In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import unet
from malaya_speech.utils import tf_featurization
import malaya_speech.augmentation.waveform as augmentation
import IPython.display as ipd
import numpy as np
from malaya_speech.utils.tf_featurization import separation_exponent, EPSILON


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
class Model:
    def __init__(self, size = 2):
        self.X = tf.placeholder(tf.float32, (None))
        self.Y = tf.placeholder(tf.float32, (size, None))
        
        stft_X, D_X = tf_featurization.get_stft(self.X)
        
        self.stft = []
        for i in range(size):
            self.stft.append(tf_featurization.get_stft(self.Y[i]))
        
        self.outputs = []
        for i in range(size):
            with tf.variable_scope(f'model_{i}'):
                self.outputs.append(unet.Model(D_X).logits)
        
        self.loss = []
        for i in range(size):
            self.loss.append(
                tf.reduce_mean(tf.abs(self.outputs[i] - self.stft[i][1]))
            )

        self.cost = tf.reduce_sum(self.loss)

In [4]:
tf.compat.v1.reset_default_graph()
model = Model()
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
var_lists = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, 'noise-reduction-unet/model.ckpt-500000')

INFO:tensorflow:Restoring parameters from noise-reduction-unet/model.ckpt-500000


In [6]:
import pickle

with open('test-set-noise-reduction.pkl', 'rb') as fopen:
    results = pickle.load(fopen)

In [7]:
results[0]

(array([ 0.04715744,  0.03726677,  0.0389775 , ..., -0.06868765,
        -0.06979144, -0.05400571]),
 array([ 0.02517777,  0.02410962,  0.02356029, ..., -0.09015168,
        -0.08832057, -0.0868862 ]),
 array([ 2.79740543e-02,  1.88972249e-02,  2.10265001e-02, ...,
         4.84924156e-07, -2.49846227e-03,  1.21944008e-02]))

In [8]:
sess.run([model.cost, model.loss], feed_dict = {model.X: results[0][0],
                                               model.Y: results[0][1:]})

[0.76937425, [0.57742643, 0.19194783]]

In [9]:
from tqdm import tqdm

total, voice, noise = [], [], []

for i in tqdm(range(len(results))):
    c, l = sess.run([model.cost, model.loss], feed_dict = {model.X: results[i][0],
                                               model.Y: results[i][1:]})
    total.append(c)
    voice.append(l[0])
    noise.append(l[1])

100%|██████████| 97/97 [00:08<00:00, 11.80it/s]


In [10]:
np.mean(total)

1.0017848

In [11]:
np.mean(voice)

0.70453936

In [13]:
np.mean(noise)

0.29724538